<a href="https://colab.research.google.com/github/SiHanWang2Developer/data/blob/master/mypointrend%E5%AE%9E%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install detectron2

查询GPU运行状态

In [0]:
!/opt/bin/nvidia-smi

Fri Mar 27 14:59:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    75W / 149W |    802MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# install dependencies: 安装依赖项
# (use +cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__

In [0]:
# install detectron2: 安装detectron2 
!pip install detectron2==0.1.1 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html
# clone the repo to access PointRend code.  克隆detectron2获取pointrend代码Use the same version as the installed detectron2
# !git clone --branch v0.1.1 https://github.com/facebookresearch/detectron2 detectron2_repo

连接Google Drive 
加载 数据集 和 detectron2_repo

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
# 您可能需要在此之前重新启动runtime，以使安装生效
# 一些基本设置：
# 设置detectron2记录器
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries导入一些常见的库
import numpy as np
import cv2
import torch
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities导入一些常见的detectron2包
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
coco_metadata = MetadataCatalog.get("coco_2017_val")

# import PointRend project  导入PointRend项目
import sys; sys.path.insert(1, "detectron2_repo/projects/PointRend")
import point_rend

## 运行预训练的PointRend模型

//   ![替代文字](https://)我们首先从COCO数据集中下载图像：

In [0]:
!wget http://images.cocodataset.org/val2017/000000005477.jpg -q -O input.jpg
im = cv2.imread("./input.jpg")
cv2_imshow(im)

加载私人数据集 下载图像

In [0]:
# !wget http://images.cocodataset.org/val2017/000000005477.jpg -q -O input.jpg
input.jpg="./drive/My Drive/......[具体路径]"
im = cv2.imread("./input.jpg")
cv2_imshow(im)

然后，我们创建一个detectron2 config 和一个detectron2 DefaultPredictor以在该mask上运行推理。

首先，我们使用标准的Mask R-CNN模型进行预测。

In [0]:
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model为此模型设置阀值
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
# mask_rcnn_predictor = DefaultPredictor(cfg)
# mask_rcnn_outputs = mask_rcnn_predictor(im)

现在，我们加载一个PointRend模型并显示其预测。

In [0]:
cfg = get_cfg()
# Add PointRend-specific config添加PointRend特定的配置
point_rend.add_pointrend_config(cfg)
# Load a config from file从文件加载配置
cfg.merge_from_file("[具体路径]detectron2_repo/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_R_50_FPN_3x_coco.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model为此模型设置阀值
# Use a model from PointRend model zoo:使用PointRend model zoo中的模型 https://github.com/facebookresearch/detectron2/tree/master/projects/PointRend#pretrained-models
cfg.MODEL.WEIGHTS = "https://dl.fbaipublicfiles.com/detectron2/PointRend/InstanceSegmentation/pointrend_rcnn_R_50_FPN_3x_coco/164955410/model_final_3c3198.pkl"
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [0]:
# Show and compare two predictions: 显示并比较两个预测
# v = Visualizer(im[:, :, ::-1], coco_metadata, scale=1.2, instance_mode=ColorMode.IMAGE_BW)
# mask_rcnn_result = v.draw_instance_predictions(mask_rcnn_outputs["instances"].to("cpu")).get_image()
v = Visualizer(im[:, :, ::-1], coco_metadata, scale=1.2, instance_mode=ColorMode.IMAGE_BW)
point_rend_result = v.draw_instance_predictions(outputs["instances"].to("cpu")).get_image()
print("Mask R-CNN with PointRend (top)     vs.     Default Mask R-CNN (bottom)")
# cv2_imshow(np.concatenate((point_rend_result, mask_rcnn_result), axis=0)[:, :, ::-1])
cv2_imshow((（point_rend_result）axis=0）[:, :, ::-1])

## 训练PointRend模型